In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

human_num=7


In [3]:
def AP(pred_list, ground_truth):
    hits = 0
    sum_prec = 0
    for i in range(len(pred_list)):
        if pred_list[i] in ground_truth.keys():
            hits += ground_truth[pred_list[i]]/human_num
            sum_prec += hits / (i + 1)
    if hits == 0:
        return 0
    return sum_prec / len(ground_truth)


def NDCG(pred_list, ground_truth):
    dcg = 0
    idcg = 0
    sorted_truth = sorted(
        ground_truth.items(), key=lambda x: x[1], reverse=True)
    for i in range(len(pred_list)):
        if pred_list[i] in ground_truth.keys():
            dcg += ground_truth[pred_list[i]] / np.log2(i + 2)
        idcg += sorted_truth[i][1] / np.log2(i + 2)
    if dcg == 0:
        return 0
    return dcg / idcg


def top_k(pred_list, ground_truth, k):
    pred_list = pred_list[:k]
    score = []
    for i in range(len(pred_list)):
        if pred_list[i] in ground_truth.keys():
            score.append(ground_truth[pred_list[i]]/human_num)
        else:
            score.append(0)
    if score:
        return np.mean(score)
    else:
        return 0


In [4]:
ground_truth = {
    'MySQL_Handlers__external_lock': 6,
    'MySQL_Handlers__write': 6,
    'MySQL_Network_Traffic__Outbound': 1,
    'MySQL_Transaction_Handlers__commit': 6,
    'MySQL_File_Openings': 4,
    'MySQL_Handlers__read_rnd': 4,
    'MySQL_Network_Usage_Hourly__Received': 1,
    'MySQL_Network_Usage_Hourly__Sent': 1,
    'MySQL_Query_Cache_Memory__Free_Memory': 2,
    'Node_Swap_Activity__Swap_In_Reads': 2,
    'Node_IO_Activity__Page_In': 5,
    'Node_Memory_Distribution__Free': 4,
    'MySQL_Thread_Cache__Threads_Created': 1
}

PerfCE = [
    'MySQL_Handlers__external_lock',
    'MySQL_Handlers__write',
    'MySQL_Network_Traffic__Outbound',
    'MySQL_Transaction_Handlers__commit'
]
CauseInfer = [
    'MySQL_Aborted_Connections__attempts',
    'Node_Memory_Distribution__Free',
    'MySQL_Thread_Cache__Threads_Created',
    'Top_Command_Counters__show_slave_hosts',
    'Top_Command_Counters__admin_commands',
]
CauseInfer_CE = [
    'MySQL_Query_Cache_Memory__Free_Memory',
    'Node_Swap_Activity__Swap_In_Reads',
    'Node_IO_Activity__Page_In',
    'Node_Memory_Distribution__Free',
    'MySQL_Connections__Max_Used_Connections',
]
FluxInfer = [
    'MySQL_File_Openings', 
    'MySQL_Handlers__read_rnd', 
    'MySQL_Network_Usage_Hourly__Received', 
    'MySQL_Network_Usage_Hourly__Sent', 
    'MySQL_Query_Cache_Memory__Free_Memory'
]
ExplainIt = [
    'MySQL_Aborted_Connections__attempts',
    'MySQL_Thread_Cache__Threads_Created',
    'Top_Command_Counters__admin_commands',
    'Node_Memory_Distribution__Free',
    'Top_Command_Counters__show_slave_hosts'
]


methods_dict = {
    'PerfCE': PerfCE,
    'CauseInfer': CauseInfer,
    'CauseInfer_CE': CauseInfer_CE,
    'FluxInfer': FluxInfer,
    'ExplainIt': ExplainIt
}

In [ ]:
for k,v in methods_dict.items():
    
    print(f'Top-1 for {k}: {top_k(v, ground_truth, 1)}')
    print(f'Top-3 for {k}: {top_k(v, ground_truth, 3)}')
    print(f'Top-5 for {k}: {top_k(v, ground_truth, 5)}')
    print(f'MAP for {k}: {AP(v, ground_truth)}')
    print(f'NDCG for {k}: {NDCG(v, ground_truth)}')

    print('==========================================================')

In [61]:
for k,v in ground_truth.items():
    print(f"{k}:{v};", end="")
print(',', end="")
for i in PerfCE:
    print(f"{i};", end="")
print(',', end="")
for i in CauseInfer:
    print(f"{i};", end="")
print(',', end="")
for i in CauseInfer_CE:
    print(f"{i};", end="")
print(',', end="")
for i in FluxInfer:
    print(f"{i};", end="")
print(',', end="")
for i in ExplainIt:
    print(f"{i};", end="")


MySQL_Handlers__external_lock:6;MySQL_Handlers__write:6;MySQL_Network_Traffic__Outbound:1;MySQL_Transaction_Handlers__commit:6;MySQL_File_Openings:4;MySQL_Handlers__read_rnd:4;MySQL_Network_Usage_Hourly__Received:1;MySQL_Network_Usage_Hourly__Sent:1;MySQL_Query_Cache_Memory__Free_Memory:2;Node_Swap_Activity__Swap_In_Reads:2;Node_IO_Activity__Page_In:5;Node_Memory_Distribution__Free:4;MySQL_Thread_Cache__Threads_Created:1;,MySQL_Handlers__external_lock;MySQL_Handlers__write;MySQL_Network_Traffic__Outbound;MySQL_Transaction_Handlers__commit;,MySQL_Aborted_Connections__attempts;Node_Memory_Distribution__Free;MySQL_Thread_Cache__Threads_Created;Top_Command_Counters__show_slave_hosts;Top_Command_Counters__admin_commands;,MySQL_Query_Cache_Memory__Free_Memory;Node_Swap_Activity__Swap_In_Reads;Node_IO_Activity__Page_In;Node_Memory_Distribution__Free;MySQL_Connections__Max_Used_Connections;,MySQL_File_Openings;MySQL_Handlers__read_rnd;MySQL_Network_Usage_Hourly__Received;MySQL_Network_Usage_Ho

In [ ]:
eval_df=pd.read_csv('/home/zjiae/Project/TiDB_exp/src/mysql/eval.csv')

In [45]:
case_no = 12

eval_df = pd.read_csv('/home/zjiae/Project/TiDB_exp/src/mysql/eval.csv')
print(eval_df.loc[case_no, 'case'], '\n')
truth_line = eval_df.loc[case_no, 'ground_truth'].split(';')
ground_truth = {}
for i in truth_line:
    if i:
        ground_truth[i.split(':')[0]] = int(i.split(':')[1])
for k, v in ground_truth.items():
    print(f"{k}:{v}")

ExplainIt = [
    'Top_Command_Counters__admin_commands',
    'Top_Command_Counters__show_slave_hosts',
    'MySQL_Thread_Cache__Threads_Created',
    'MySQL_Aborted_Connections__attempts',
    'Node_Memory_Distribution__Free',
]
PerfCE = list(filter(None, eval_df.loc[case_no, 'PerfCE'].split(';')))
CauseInfer = list(filter(None, eval_df.loc[case_no, 'CauseInfer'].split(';')))
CauseInfer_CE = list(filter(None, eval_df.loc[case_no, 'CauseInfer_CE'].split(';')))
FluxInfer = list(filter(None, eval_df.loc[case_no, 'FluxInfer'].split(';')))

methods_dict = {
    'PerfCE': PerfCE,
    'CauseInfer': CauseInfer,
    'CauseInfer_CE': CauseInfer_CE,
    'FluxInfer': FluxInfer,
    'ExplainIt': ExplainIt
}


Flush_Tables 

MySQL_Handlers__external_lock:6
MySQL_Handlers__write:6
MySQL_Network_Traffic__Outbound:1
MySQL_Transaction_Handlers__commit:6
MySQL_File_Openings:4
MySQL_Handlers__read_rnd:4
MySQL_Network_Usage_Hourly__Received:1
MySQL_Network_Usage_Hourly__Sent:1
MySQL_Query_Cache_Memory__Free_Memory:2
Node_Swap_Activity__Swap_In_Reads:2
Node_IO_Activity__Page_In:5
Node_Memory_Distribution__Free:4
MySQL_Thread_Cache__Threads_Created:1


In [46]:
print(f'Top-1: {top_k(ExplainIt, ground_truth, 1)}')
print(f'Top-3: {top_k(ExplainIt, ground_truth, 3)}')
print(f'Top-5: {top_k(ExplainIt, ground_truth, 5)}')
print(f'MAP: {AP(ExplainIt, ground_truth)}')
print(f'NDCG: {NDCG(ExplainIt, ground_truth)}')

Top-1: 0.0
Top-3: 0.047619047619047616
Top-5: 0.14285714285714285
MAP: 0.014652014652014652
NDCG: 0.1241880967980982


In [14]:
ground_truth = {
    'Node_Memory_Distribution__Free': 4,
    'MySQL_Query_Cache_Memory__Free_Memory': 2,
    'Node_CPU_Usage_Load__user': 7,
    'Node_CPU_Usage_Load__Load_1m': 7,
    'Node_CPU_Usage_Load__Max_Core_Utilization': 7,
    'Node_IO_Activity__Page_Out': 4,
    'Node_IO_Activity__Page_In': 4,
    'MySQL_Network_Usage_Hourly__Sent': 2,
}

PerfCE=[
    'MySQL_Network_Usage_Hourly__Sent',
    'Node_CPU_Usage_Load__Load_1m',
    'Node_IO_Activity__Page_Out',
    'Node_IO_Activity__Page_In',
]

print(NDCG(PerfCE, ground_truth))
print(AP(PerfCE, ground_truth))
print(top_k(PerfCE, ground_truth, 3))
print(top_k(PerfCE, ground_truth, 5))

0.6093565596410374
0.2693452380952381
0.619047619047619
0.6071428571428571
